# FAKE REVIEW NODE EMBEDDING

In [412]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [413]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [414]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [415]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [416]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [417]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

In [418]:
dgl.__version__

'0.7.2'

In [419]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [420]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [421]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [422]:
#loading dataset
df=pd.read_csv('/content/category 4.csv')
df.head()

,category,rating,label,text_
0,Pet_Supplies_5,5.0,CG,Ears were really bad and had an odor. I was concerned about my dogs getting sick and
1,Pet_Supplies_5,5.0,CG,"Dog loves them, I have a 9 year old Lab and I have a 5 month old Lab and"
2,Pet_Supplies_5,1.0,CG,"I used them, but they are not as good as they used to be, and I don't"
3,Pet_Supplies_5,5.0,CG,My dog loved them and it has kept her occupied for a long time. She does not mind
4,Pet_Supplies_5,5.0,CG,My two cats love Hill's treats. The only problem is that they are kind of hard to find


In [423]:
#df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [424]:
df.shape

(4254, 4)

In [425]:
df['label'].value_counts()

CG    2127
OR    2127
Name: label, dtype: int64

In [426]:
# dropping unnecessary columns. 
df.drop(['category', 'rating'], axis = 1, inplace=True)
df.head()

,label,text_
0,CG,Ears were really bad and had an odor. I was concerned about my dogs getting sick and
1,CG,"Dog loves them, I have a 9 year old Lab and I have a 5 month old Lab and"
2,CG,"I used them, but they are not as good as they used to be, and I don't"
3,CG,My dog loved them and it has kept her occupied for a long time. She does not mind
4,CG,My two cats love Hill's treats. The only problem is that they are kind of hard to find


In [427]:
z = {'OR' : 1, 'CG' : 0}  # 1 for original reviews and 0 for computer generated reviews
df['label'] = df['label'].map(z)

In [428]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [429]:
# applying that into function.
df["clean_text"] = df['text_'].apply(lambda x: clean_text(x))
df.head(10)

,label,text_,clean_text
0,0,Ears were really bad and had an odor. I was concerned about my dogs getting sick and,Ears were really bad and had an odor I was concerned about my dogs getting sick and
1,0,"Dog loves them, I have a 9 year old Lab and I have a 5 month old Lab and",Dog loves them I have a year old Lab and I have a month old Lab and
2,0,"I used them, but they are not as good as they used to be, and I don't",I used them but they are not as good as they used to be and I don t
3,0,My dog loved them and it has kept her occupied for a long time. She does not mind,My dog loved them and it has kept her occupied for a long time She does not mind
4,0,My two cats love Hill's treats. The only problem is that they are kind of hard to find,My two cats love Hill s treats The only problem is that they are kind of hard to find
5,0,"it was cracked a little, but the pieces were fine.\n\nThe only reason I gave it",it was cracked a little but the pieces were fine The only reason I gave it
6,0,Not worth the cost. We have 3 dogs and they all love it! I have one,Not worth the cost We have dogs and they all love it I have one
7,0,I like it but my dog has a sensitive stomach. I will not buy it again.,I like it but my dog has a sensitive stomach I will not buy it again
8,0,"works vary good my first time using it. I've tried other brands, but this one is",works vary good my first time using it I ve tried other brands but this one is
9,0,Great for my small picky dog. The only problem is that it's kind of hard to put,Great for my small picky dog The only problem is that it s kind of hard to put


In [430]:
# dropping the uncleaned text column "text"
df = df.drop(['text_'], axis=1)

In [431]:
# renaming the column
df.rename(columns = {'clean_text':'text', 'label': 'Class'}, inplace = True)
df.head()

,Class,text
0,0,Ears were really bad and had an odor I was concerned about my dogs getting sick and
1,0,Dog loves them I have a year old Lab and I have a month old Lab and
2,0,I used them but they are not as good as they used to be and I don t
3,0,My dog loved them and it has kept her occupied for a long time She does not mind
4,0,My two cats love Hill s treats The only problem is that they are kind of hard to find


In [432]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'’m', '‘d', 'why', 'off', 'this', 'one', 'enough', 'these', 'could', 'fifteen', 'yourself', 'ten', 'move', 'whom', 'seems', 'mostly', 'almost', 'when', 'its', 'rather', 'before', 'wherein', 'or', 'somewhere', 'therein', 'twenty', 'me', '’d', 'except', 'were', 'either', 'anyhow', 'most', 'whence', 'moreover', 'she', 'everywhere', 'her', 'throughout', 'alone', 'latterly', 'six', 'sometime', 'against', 'hers', 'top', 'after', 'as', 'above', 'besides', 'put', 'seemed', 'via', 'side', "n't", 'your', 'without', 'whatever', 'whose', 'ca', 'yours', '‘s', 'very', 'regarding', 'already', 'whither', 'is', 'wherever', 'fifty', 'others', 'more', 'behind', 'thru', 'was', 'now', 'where', 'few', 'along', 'once', 'thence', 'forty', 'he', 'whereby', '’s', '‘ve', 'it', 'three', 'and', 'get', 'would', 'both', 'own', 'hereupon', 'herself', 'made', 'us', 'else', 'if', 'around', 'seeming', 'used', '’ve', 'even', 'myself', 'n’t', 'an', 'otherwise', 'the', 'then', 'upon', 'something', 'than', 'least', 'only',

In [433]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [434]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [435]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [436]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,ear bad odor concerned dog getting sick
1,0,dog love year old lab month old lab
2,0,good
3,0,dog loved kept occupied long time mind
4,0,cat love hill treat problem kind hard find


### FEATURE EXTRACTION

In [437]:
x = df['text']
y = df['Class']

In [438]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [439]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [440]:
len(feature_names)

7239

In [441]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aa,aaronco,ab,abandoned,abated,abdomen,abdominal,ability,able,aboard,...,zipper,zippered,zone,zoo,zoom,zoomies,zucchini,zuke,zx,zymox
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [442]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 700)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [443]:
chi_support = chi2_features.get_support()

In [444]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [445]:
chi_feature

['accident',
 'active',
 'actually',
 'add',
 'addition',
 'adjustable',
 'adjustment',
 'adverse',
 'advertised',
 'african',
 'aggressive',
 'air',
 'allergy',
 'alternative',
 'amazed',
 'amazon',
 'animal',
 'annoyed',
 'annoying',
 'anti',
 'apparently',
 'appears',
 'applying',
 'appreciate',
 'area',
 'arm',
 'ask',
 'assembly',
 'attached',
 'auto',
 'available',
 'average',
 'away',
 'awhile',
 'baby',
 'bacteria',
 'ball',
 'barely',
 'bat',
 'batch',
 'bath',
 'bathing',
 'battery',
 'bc',
 'beagle',
 'beautifully',
 'bed',
 'begin',
 'bendable',
 'best',
 'better',
 'big',
 'bill',
 'bird',
 'bit',
 'bite',
 'black',
 'body',
 'bone',
 'boot',
 'bother',
 'bought',
 'bowl',
 'box',
 'breath',
 'breeder',
 'bring',
 'brown',
 'brush',
 'brushing',
 'bubble',
 'buckle',
 'buffalo',
 'build',
 'bull',
 'bunny',
 'burn',
 'butter',
 'button',
 'buyer',
 'buying',
 'came',
 'canned',
 'cant',
 'cap',
 'car',
 'carbon',
 'care',
 'carefully',
 'carry',
 'carrying',
 'case',
 'cat

In [446]:
X_kbest_features

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.05468767, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [447]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,accident,active,actually,add,addition,adjustable,adjustment,adverse,advertised,african,...,wish,wonderfully,work,working,world,writing,year,yes,younger,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.258513,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [448]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(X_kbest_features)

In [449]:
src

array([   0,    0,    0, ..., 4253, 4253, 4253])

In [450]:
dst

array([124, 160, 175, ..., 661, 667, 678])

In [451]:
g = dgl.graph((src, dst))

In [452]:
n_nodes = g.num_nodes()
n_nodes

4254

In [453]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 4251, 4252, 4253])

In [454]:
n_nodes1 = g.number_of_nodes()
n_nodes1

4254

In [455]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0547,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [456]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [457]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [458]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [459]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [460]:
train_mask[:n_train] = True

In [461]:
val_mask[n_train:n_train + n_val] = True

In [462]:
test_mask[n_train + n_val:] = True

In [463]:
g.ndata['train_mask'] = train_mask

In [464]:
g.ndata['val_mask'] = val_mask

In [465]:
g.ndata['test_mask'] = test_mask

In [466]:
s_array = df['Class'].to_numpy()
s_array

array([0, 0, 0, ..., 1, 0, 1])

In [467]:
t1 = torch.tensor(s_array)
t1

tensor([0, 0, 0,  ..., 1, 0, 1])

In [468]:
g.ndata["label"] =t1

In [469]:
label = g.ndata["label"]

In [470]:
label[train_mask]

tensor([0, 0, 0,  ..., 0, 1, 1])

In [471]:
label[val_mask]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,

In [472]:
label[test_mask]

tensor([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,

In [473]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [474]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [475]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [476]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

In [477]:
accuracy_score(label[val_mask], y_val_lr)

0.8858823529411765

In [478]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])
accuracy_score(label[val_mask], y_val_lrb)

0.8858823529411765

In [479]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8035294117647059

In [480]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.76

In [481]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.7529411764705882

In [482]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=7, shuffle=True)

In [483]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8573117923225629


In [484]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8739994476663906


In [485]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8516575531621099


In [486]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8293327809997239


In [487]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=300)
scores = cross_val_score(ab, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.825818834576084


In [488]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=300)
scores = cross_val_score(gb, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.846028721347694


In [489]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8486147473073735


In [490]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [491]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [492]:
node_features

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0547,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [493]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [494]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, test_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.5316901408450704
0.5845070422535211


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0.6244131455399061
0.6643192488262911
0.6854460093896714
0.7206572769953051
0.7535211267605634
0.7875586854460094
0.7934272300469484
0.8098591549295775
0.8227699530516432
0.8380281690140845
0.852112676056338
0.8603286384976526
0.8650234741784038
0.8708920187793427
0.886150234741784
0.886150234741784
0.8884976525821596
0.8943661971830986
0.897887323943662
0.9002347417840375
0.9025821596244131
0.9061032863849765
0.9096244131455399
0.9143192488262911
0.9166666666666666
0.9178403755868545
0.9225352112676056
0.9248826291079812
0.9272300469483568
0.9307511737089202
0.9307511737089202
0.9295774647887324
0.931924882629108
0.9330985915492958
0.9330985915492958
0.9366197183098591
0.9401408450704225
0.9424882629107981
0.9413145539906104
0.9424882629107981
0.9413145539906104
0.9401408450704225
0.9413145539906104
0.9401408450704225
0.9424882629107981
0.9413145539906104
0.9401408450704225
0.9401408450704225
